<a href="https://colab.research.google.com/github/youbodib/LeNet-5/blob/main/LeNet_5_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random

Chargement & Split du Dataset

In [22]:
def load_dataset(data_root):
    X, y, class_names = [], [], sorted([d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))])
    for label, class_dir in enumerate(class_names):
        class_path = os.path.join(data_root, class_dir)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = Image.open(img_path).convert('L').resize((32,32))
                arr = np.asarray(img, dtype=np.float32) / 255.0
                arr = arr[np.newaxis, ...]  # shape (1,32,32)
                X.append(arr)
                y.append(label)
            except Exception as e:
                continue
    X = np.stack(X, axis=0)
    y = np.array(y, dtype=np.int32)
    return X, y, class_names

DATA_ROOT = './tifinagh-data/amhcd-data-64/tifinagh-images'
X, y, class_names = load_dataset(DATA_ROOT)
perm = np.random.permutation(len(X))
n_train = int(0.8 * len(X))
X_train, y_train = X[perm[:n_train]], y[perm[:n_train]]
X_test, y_test = X[perm[n_train:]], y[perm[n_train:]]

Data Augmentation

In [23]:
def augment_batch(X):
    X_aug = X.copy()
    for i in range(X.shape[0]):
        if random.random() < 0.5:
            X_aug[i] = np.flip(X_aug[i], axis=2)  # horizontal flip
        X_aug[i] += np.random.normal(0, 0.02, X_aug[i].shape)
        X_aug[i] = np.clip(X_aug[i], 0, 1)
    return X_aug

Couches From Scratch
- Convolution *2D*

In [24]:
class Conv2D:
    def __init__(self, in_ch, out_ch, kernel_size):
        self.in_ch = in_ch
        self.out_ch = out_ch
        self.kernel_size = kernel_size
        scale = np.sqrt(1. / (in_ch * kernel_size * kernel_size))
        self.W = np.random.randn(out_ch, in_ch, kernel_size, kernel_size) * scale
        self.b = np.zeros((out_ch, 1))
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

    def forward(self, x):
        self.x = x
        B, C, H, W = x.shape
        out_h = H - self.kernel_size + 1
        out_w = W - self.kernel_size + 1
        y = np.zeros((B, self.out_ch, out_h, out_w))
        for n in range(B):
            for oc in range(self.out_ch):
                for ic in range(self.in_ch):
                    for i in range(out_h):
                        for j in range(out_w):
                            y[n, oc, i, j] += np.sum(
                                x[n, ic, i:i+self.kernel_size, j:j+self.kernel_size] * self.W[oc, ic]
                            )
                y[n, oc] += self.b[oc]
        return y

    def backward(self, grad_y, lr):
        x = self.x
        B, C, H, W = x.shape
        _, out_ch, out_h, out_w = grad_y.shape
        self.dW.fill(0)
        self.db.fill(0)
        dx = np.zeros_like(x)
        for n in range(B):
            for oc in range(self.out_ch):
                for ic in range(self.in_ch):
                    for i in range(out_h):
                        for j in range(out_w):
                            patch = x[n, ic, i:i+self.kernel_size, j:j+self.kernel_size]
                            self.dW[oc, ic] += grad_y[n, oc, i, j] * patch
                            dx[n, ic, i:i+self.kernel_size, j:j+self.kernel_size] += grad_y[n, oc, i, j] * self.W[oc, ic]
                self.db[oc] += np.sum(grad_y[n, oc])
        self.W -= lr * self.dW / B
        self.b -= lr * self.db / B
        return dx

- Average Pooling 2D

In [25]:
class AvgPool2D:
    def __init__(self, kernel, stride):
        self.kernel = kernel
        self.stride = stride

    def forward(self, x):
        self.x = x
        B, C, H, W = x.shape
        out_h = (H - self.kernel) // self.stride + 1
        out_w = (W - self.kernel) // self.stride + 1
        y = np.zeros((B, C, out_h, out_w))
        for n in range(B):
            for c in range(C):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start = i * self.stride
                        w_start = j * self.stride
                        y[n, c, i, j] = np.mean(x[n, c, h_start:h_start+self.kernel, w_start:w_start+self.kernel])
        return y

    def backward(self, grad_y, lr):
        x = self.x
        B, C, H, W = x.shape
        dx = np.zeros_like(x)
        out_h, out_w = grad_y.shape[2], grad_y.shape[3]
        for n in range(B):
            for c in range(C):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start = i * self.stride
                        w_start = j * self.stride
                        dx[n, c, h_start:h_start+self.kernel, w_start:w_start+self.kernel] += grad_y[n, c, i, j] / (self.kernel*self.kernel)
        return dx

- Dense Layer

In [26]:
class Dense:
    def __init__(self, in_dim, out_dim):
        scale = np.sqrt(1. / in_dim)
        self.W = np.random.randn(in_dim, out_dim) * scale
        self.b = np.zeros(out_dim)
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

    def forward(self, x):
        self.x = x
        return x @ self.W + self.b

    def backward(self, grad_y, lr):
        self.dW = self.x.T @ grad_y
        self.db = grad_y.sum(axis=0)
        dx = grad_y @ self.W.T
        self.W -= lr * self.dW / grad_y.shape[0]
        self.b -= lr * self.db / grad_y.shape[0]
        return dx

- Flatten

In [27]:
class Flatten:
    def forward(self, x):
        self.input_shape = x.shape
        return x.reshape(x.shape[0], -1)
    def backward(self, grad_y, lr):
        return grad_y.reshape(self.input_shape)

- ReLU

In [28]:
class ReLU:
    def forward(self, x):
        self.mask = (x > 0)
        return x * self.mask
    def backward(self, grad_y, lr):
        return grad_y * self.mask

Softmax & Cross-Entropy

In [29]:
def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

def cross_entropy(probs, y_true):
    N = y_true.shape[0]
    return -np.log(probs[range(N), y_true] + 1e-9).mean()

def grad_softmax_crossentropy(probs, y_true):
    N = y_true.shape[0]
    grad = probs.copy()
    grad[range(N), y_true] -= 1
    grad /= N
    return grad

LeNet-5 Architecture (forward & backward)

In [30]:
class LeNet5Scratch:
    def __init__(self, num_classes):
        self.conv1 = Conv2D(1, 6, 5)
        self.relu1 = ReLU()
        self.pool1 = AvgPool2D(2, 2)
        self.conv2 = Conv2D(6, 16, 5)
        self.relu2 = ReLU()
        self.pool2 = AvgPool2D(2, 2)
        self.conv3 = Conv2D(16, 120, 5)
        self.relu3 = ReLU()
        self.flatten = Flatten()
        self.fc1 = Dense(120, 84)
        self.relu4 = ReLU()
        self.fc2 = Dense(84, num_classes)

    def forward(self, x):
        x = self.conv1.forward(x)
        x = self.relu1.forward(x)
        x = self.pool1.forward(x)
        x = self.conv2.forward(x)
        x = self.relu2.forward(x)
        x = self.pool2.forward(x)
        x = self.conv3.forward(x)
        x = self.relu3.forward(x)
        x = self.flatten.forward(x)
        x = self.fc1.forward(x)
        x = self.relu4.forward(x)
        x = self.fc2.forward(x)
        return x

    def backward(self, grad, lr):
        grad = self.fc2.backward(grad, lr)
        grad = self.relu4.backward(grad, lr)
        grad = self.fc1.backward(grad, lr)
        grad = self.flatten.backward(grad, lr)
        grad = self.relu3.backward(grad, lr)
        grad = self.conv3.backward(grad, lr)
        grad = self.pool2.backward(grad, lr)
        grad = self.relu2.backward(grad, lr)
        grad = self.conv2.backward(grad, lr)
        grad = self.pool1.backward(grad, lr)
        grad = self.relu1.backward(grad, lr)
        grad = self.conv1.backward(grad, lr)
        return grad

Boucle d’entraînement

In [31]:
def train(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32, lr=0.01):
    N = X_train.shape[0]
    for epoch in range(epochs):
        perm = np.random.permutation(N)
        X_train, y_train = X_train[perm], y_train[perm]
        train_losses, train_accs = [], []
        for i in range(0, N, batch_size):
            Xb = X_train[i:i+batch_size]
            yb = y_train[i:i+batch_size]
            # Xb = augment_batch(Xb)  # décommente pour data augmentation
            logits = model.forward(Xb)
            probs = softmax(logits)
            loss = cross_entropy(probs, yb)
            preds = np.argmax(probs, axis=1)
            acc = np.mean(preds == yb)
            train_losses.append(loss)
            train_accs.append(acc)
            grad = grad_softmax_crossentropy(probs, yb)
            model.backward(grad, lr)
        val_acc = evaluate(model, X_val, y_val, batch_size)
        print(f"Epoch {epoch+1} | Loss: {np.mean(train_losses):.4f} | Train acc: {np.mean(train_accs):.4f} | Val acc: {val_acc:.4f}")

def evaluate(model, X, y, batch_size=32):
    N = X.shape[0]
    accs = []
    for i in range(0, N, batch_size):
        Xb = X[i:i+batch_size]
        yb = y[i:i+batch_size]
        logits = model.forward(Xb)
        probs = softmax(logits)
        preds = np.argmax(probs, axis=1)
        acc = np.mean(preds == yb)
        accs.append(acc)
    return np.mean(accs)

Lancement de l’entraînement

In [ ]:
model = LeNet5Scratch(num_classes=len(class_names))
train(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32, lr=0.01)

Prédiction et Affichage

In [ ]:
# Afficher 10 images test et leur prédiction
idxs = np.random.choice(len(X_test), 10, replace=False)
plt.figure(figsize=(15,3))
for i, idx in enumerate(idxs):
    img = X_test[idx][0]
    label = y_test[idx]
    pred = np.argmax(softmax(model.forward(X_test[idx:idx+1])))
    plt.subplot(1,10,i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"T:{class_names[label]}\nP:{class_names[pred]}", fontsize=8)
    plt.axis('off')
plt.show()

Matrice de confusion

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
all_preds = np.argmax(softmax(model.forward(X_test)), axis=1)
cm = confusion_matrix(y_test, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
plt.figure(figsize=(10,10))
disp.plot(xticks_rotation=90, cmap='Blues')
plt.show()